In [ ]:
!ls

In [1]:
import sqlite3
import pandas as pd
from dateutil.relativedelta import relativedelta
import numpy as np

In [ ]:
DATES_COLS = ['date_start', 'date_end']

In [ ]:

# Create your connection.
cnx = sqlite3.connect('../db/db.db')

df = pd.read_sql_query("SELECT * FROM premium_payments", cnx, parse_dates=DATES_COLS)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df['date_start'] = pd.to_datetime(df['date_start'])
df['date_end'] = pd.to_datetime(df['date_end'])

In [ ]:
df

In [ ]:
df.member_id.value_counts()

In [ ]:
# id 1553 get differents type of premium

In [ ]:
df[df['member_id'] == 1553]

In [ ]:
sample = df[df['member_id'] == 1553]

In [ ]:
sample

In [ ]:
sample['test'] = sample.groupby(['member_id', 'type'])['date_end'].shift(1)

In [ ]:
sample

In [ ]:
sample['date_start'].min()

In [ ]:
#df.merge(df.groupby(['C', 'D']).apply(lambda row: row['B'] - row['A']).sum(level=[0,1]).reset_index())

In [ ]:
sample.groupby(['member_id', 'type']).apply(lambda row: row['date_end'].max()-row['date_start'].min())

In [ ]:
sample.groupby(['member_id', 'type']).transform(lambda row: row['date_end'].max()-row['date_start'].min())

In [ ]:
result = sample.groupby(['member_id', 'type']).apply(lambda row: row['date_end'].max()-row['date_start'].min())

In [ ]:
result

In [ ]:
sample['relativedelta'] = 

In [ ]:
sample['transaction_class'] = np.where(sample['date_start']+pd.Timedelta(3, unit='D')<sample['test'], 'other_transaction', 'same transaction')

In [ ]:
sample

In [ ]:
sample.groupby(['member_id', 'type', 'transaction_class']).agg({
    'date_start': min,
    'date_end': max
})

In [ ]:
sample['test_bis'] = sample['date_start'] - sample['test']

In [ ]:
sample

In [ ]:
sample['test_bis'] + pd.Timedelta(3, unit='D') 

In [ ]:
experiment = df.copy()

In [ ]:
# get number of members
df.member_id.nunique()

In [ ]:
def _group_transaction(df):
    df['end_date_alg'] = df.groupby(['member_id', 'type'])['date_end'].shift(1)
    return np.where(df['date_start']+pd.Timedelta(3, unit='D')<df['end_date_alg'], 'other_transaction', 'same transaction')

In [ ]:
df['date_start']+pd.Timedelta(3, unit='D')

In [ ]:
experiment['transaction_class'] = _group_transaction(experiment)

In [ ]:
experiment

In [ ]:
experiment.transaction_class.value_counts()

In [ ]:
result = experiment.groupby(['member_id', 'type', 'transaction_class']).agg({
    'date_start': min,
    'date_end': max
})

In [ ]:
result

In [ ]:
result.columns

In [ ]:
result.index

In [ ]:
result.reset_index(level='transaction_class').drop('transaction_class', axis=1)

In [ ]:
final_result = result.reset_index().drop('transaction_class', axis=1)

In [ ]:
final_result

In [ ]:
final_result.member_id.value_counts()

In [ ]:
final_result[final_result['member_id']==9637]

check results 

In [5]:
!ls ../etl/db

db.db             db_transformed.db


In [6]:
# Create your connection.
cnx = sqlite3.connect('../etl/db/db_transformed.db')

df = pd.read_sql_query("SELECT * FROM premium_payments_transformed", cnx)

In [7]:
df

,member_id,type,date_start,date_end
0,1,1,2018-10-31 00:00:00,2019-01-01 00:00:00
1,2,3,2019-01-18 00:00:00,2019-04-23 00:00:00
2,3,3,2019-03-30 00:00:00,2019-07-29 00:00:00
3,4,2,2019-11-29 00:00:00,2020-03-16 00:00:00
4,5,3,2019-11-28 00:00:00,2020-01-29 00:00:00
...,...,...,...,...
12720,9997,1,2019-01-07 00:00:00,2019-06-09 00:00:00
12721,9997,2,2019-02-08 00:00:00,2019-03-11 00:00:00
12722,9998,1,2019-09-03 00:00:00,2019-12-07 00:00:00
12723,9998,2,2019-06-05 00:00:00,2019-07-05 00:00:00
